In [1]:
from numpy import asarray
from numpy import expand_dims
import numpy as np
import pickle
import cv2
from architecture import * 
from mtcnn import MTCNN

In [2]:
#Create detector model
detector = MTCNN()

# Create the FaceNet model
face_encoder = InceptionResNetV2()
face_encoder.load_weights('facenet_keras.h5')

2022-07-12 11:53:46.470258: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
with open("data.pkl", "rb") as myfile:
    database = pickle.load(myfile) 
    myfile.close()

In [6]:
cap = cv2.VideoCapture(0)


OpenCV: camera access has been denied. Either run 'tccutil reset Camera' command in same terminal to reset application authorization status, either modify 'System Preferences -> Security & Privacy -> Camera' settings for your application.
OpenCV: camera failed to properly initialize!


In [5]:
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    _, image = cap.read()
    
    faces = detector.detect_faces(image)
    
    if len(faces)>0:
        x1, y1, width, height = faces[0]['box']
    else:
        x1, y1, width, height = 1, 1, 10, 10
    
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face = image[y1:y2, x1:x2]                        
    face = cv2.resize(face, (160, 160))
    face = asarray(face)
    
    face = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    
    face = expand_dims(face, axis=0)
    signature = face_encoder.predict(face)
    
    min_dist=100
    identity=' '
    for key, value in database.items():
        dist = np.linalg.norm(value-signature)
        if dist < min_dist:
            min_dist = dist
            identity = key
            
    cv2.putText(image,identity, (100,100),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
    cv2.rectangle(image,(x1,y1),(x2,y2), (0,255,0), 2)
        
    cv2.imshow('res',image)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()
cap.release()

OpenCV: camera access has been denied. Either run 'tccutil reset Camera' command in same terminal to reset application authorization status, either modify 'System Preferences -> Security & Privacy -> Camera' settings for your application.
OpenCV: camera failed to properly initialize!


InvalidImage: Image not valid.